# IMDB Database Classifications

## Fetch data

In [1]:
import tensorflow as tf
import numpy as np
import math 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=4000)

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

## Create the vocabulary

In [2]:
from collections import Counter

vocabulary = list()
train_words = list()
sorted_words = list()
for text in x_train:
  tokens = text.split()
  train_words.extend(tokens)

Counter = Counter(train_words)
Counter_copy = Counter
temp = Counter.most_common(3998)
for key in temp:
    sorted_words.append(key[0])

#n=99, m = 1000, k = 2898
k=list()
n=list()
m = Counter.most_common(1100)
j=0
for key in m:
  j+=1
  if(j>=101):
    vocabulary.append(key[0])
 

for i in range(1,100):
  n.append(sorted_words[i])
j=0
for key in sorted_words:
  j+=1
  if(j<=1100):
    sorted_words.remove(key)
k = sorted_words.copy()

## Create binary vectors 

In [3]:
from tqdm import tqdm

x_train_binary = list()
x_test_binary = list()

for text in tqdm(x_train):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_train_binary.append(binary_vector)

x_train_binary = np.array(x_train_binary)

for text in tqdm(x_test):
  tokens = text.split()
  binary_vector = list()
  for vocab_token in vocabulary:
    if vocab_token in tokens:
      binary_vector.append(1)
    else:
      binary_vector.append(0)
  x_test_binary.append(binary_vector)

x_test_binary = np.array(x_test_binary)
y_train_list = y_train.tolist()

vocabulary_indexes = list()
for i in range(len(vocabulary)):
  vocabulary_indexes.append(i)

100%|██████████| 25000/25000 [01:13<00:00, 342.01it/s]


## Naive Bayes classifier


In [4]:
class Naive_Bayes:

    def __init__(self):
        #Λίστες με τις πιθανότητες να έχουμε την κάθε λέξη δεδομένου πως έχουμε αρνητικό ή θετικό review αντίστοιχα
        self.x1_while_c_is_negative = list()
        self.x1_while_c_is_positive = list()
        #καθολικές μεταβλητές με την πιθανότητα να έχουμε αρνητικό review και θετικό review αντιστοιχα
        self.p_c0 = float(0)
        self.p_c1 = float(0)

    def fit(self, X, Y):

        reviews = len(Y) 

        #Υπολογισμός της γενικής πιθανότητας να έχουμε θετικό ή αρνητικό review:
        neg_reviews = 0 
        pos_reviews = 0
        for i in range(len(Y)):
            if Y[i] == 0:
                neg_reviews += 1
            else:
                pos_reviews += 1
        p_positive_reviews = pos_reviews/reviews
        p_negative_reviews = neg_reviews/reviews
        #Αποθήκευση του αποτελέσματος τις καθολικές μεταβλητές
        self.pc0 = p_negative_reviews
        self.pc1 = p_positive_reviews

        #Υπολογισμός πιθανοτήτων να έχουμε την κάθε λέξη δεδομένου πως έχουμε αρνητικό ή θετικό review αντίστοιχα:
        

        #Αρχικοποίηση λιστών με μετρητές
        sum_pos = list()
        sum_neg = list()
        for i in range(0,1000):
            sum_pos.append(0) 
            sum_neg.append(0)

        for i in range(len(Y)):
            if Y[i] == 0:
                j=-1
                for word in X[i]:
                    j+=1
                    if word == 1:
                        sum_neg[j] +=1
            else:
                j=-1
                for word in X[i]:
                    j+=1
                    if word == 1:
                        sum_pos[j] +=1

        #Λίστες πιθανοτήτων να έχουμε την κάθε λέξη δεδομένου πως έχουμε αρνητικό ή θετικό review 
        p_ex_pos = list()
        p_ex_neg = list()
        for i in range(0,1000):
            p_ex_pos.append(0) 
            p_ex_neg.append(0)
        #for i in range(len(p_ex_neg_train)):
        for i in range(0,1000):
            if(sum_neg[i]!=0):
                p_ex_neg[i] = sum_neg[i]/neg_reviews #P(Xi = 1 | C = 0)
            else:
                p_ex_neg[i] = -1 # -1 για τις λέξεις που δεν εμφανίζονται καν
        for i in range(0,1000):
            if(sum_pos[i]!=0):
                p_ex_pos[i] = sum_pos[i]/neg_reviews #P(Xi = 1 | C = 1)
            else:
                p_ex_pos[i] = -1 # -1 για τις λέξεις που δεν εμφανίζονται καν

        self.x1_while_c_is_positive = p_ex_pos.copy()
        self.x1_while_c_is_negative = p_ex_neg.copy()


    def predict(self, X):

        predictions = list()
        for i in range(len(X)):

            #Το pc0 Θα γίνει pc0 = P(C=0)*P(X1=x1 | C=0)*.......*P(Xn = xn| C=0)
            pc0 = (self.pc0)   
            for xi in range(len(X[i])):
                if(x_train_binary[i][xi] == 0):
                    pc0 *= (float(1)-self.x1_while_c_is_negative[xi])  
                else:
                    pc0 *= (self.x1_while_c_is_negative[xi])
            pc1 = (self.pc1)
            for xi in range(len(X[i])):
                if(x_train_binary[i][xi]==1):
                    pc1 *= self.x1_while_c_is_positive[xi]
                else:
                    pc1 *= float(1) - self.x1_while_c_is_positive[xi]
            if (pc0<pc1):
                predictions.append(1)
            else:
                predictions.append(0)

        return predictions




## ID3 Classifier

In [7]:
import math 

def IG(class_, feature):
  classes = set(class_)

  Hc = 0
  for c in classes:
    pc = list(class_).count(c)/len(class_)
    Hc += - pc * math.log(pc, 2)
  feature_values = set(feature)

  Hc_feature = 0
  for feat in feature_values:
    
    #pf --> P(X=x)
    pf = list(feature).count(feat)/len(feature)
    indices = [i for i in range(len(feature)) if feature[i] == feat]
    clasess_of_feat = [class_[i] for i in indices]
    for c in classes:
        #pcf --> P(C=c|X=x)
        pcf = clasess_of_feat.count(c)/len(clasess_of_feat)
        if pcf != 0: 
            # - P(X=x) * P(C=c|X=x) * log2(P(C=c|X=x))
            temp_H = - pf * pcf * math.log(pcf, 2)
            #sum for all values of C (class) and X (values of specific feature)
            Hc_feature += temp_H
  ig = Hc - Hc_feature
  return ig




class Tree():
    def __init__(self):
        self.word = "no word yet" #Η λέξη με την οποία θα έγινε το classification ενός υπόδεντρου
        self.tag = None #1 αν ο κόμβος έχει reviews με τη λέξη με την οποία έγινε το classification, 0 αν δεν την έχουν
        self.children = list() #τα παιδία ενός κόμβου
        self.classification = int #Η τελική classification. Παίρνει τιμή μόνο αν έχει γίνει 
    
    def new_child(self, node):
        self.children.append(node)

class ID3():
    def __init__(self, max_depth = 10):
        self.max_depth = max_depth
        self.depth = 0

    def most_IG(self, X, Y, vocabulary):

        max_gain = -1
        max_word= -1


        for w in vocabulary:
            x_word = list() #Λίστα με όλες τις τιμές που θα πάρει μια λέξη στον Χ
            for ex in range(len(X)):
                x_word.append(X[ex][w])
            word_ig = IG(Y, x_word) #Στέλνουμε το Υ και την λίστα στον ΙG, για να βρει το informtion gain της λέξης ανάλογικά με το Υ

            if(word_ig>max_gain):
                max_gain = word_ig
                max_word = w

        return max_word #Η λέξη με το μέγιστο Information Gain

    def fit(self, X, Y, vocabulary, default):
       
        if(len(Y) == 0):
            #αν φτάσαμε εδώ τελείωσαν τα Υ γιατί τελείωσε η κατάταξη κάθε review στο δέντρο.
            #Παίρνει για τιμή του classification εκείνη που επικρατούσε στο παραπάνω επίπεδο του δέντρο
            node = Tree()
            node.classification = default 
            return node 

        if(len(set(Y)) == 1):
            #Η μέθοδος set επιστρέφει ένα set με όλες τις διαφορετικές τιμές που περιλαμβάνει το όρισμα της, εδώ το Υ
            #Άρα φτάσαμε εδώ αν το Υ έχει μόνο μια τιμη, η οποία θα χρησιμοποιηθεί στο classification και σταματάει η διαδικασία. 
            node = Tree()
            node.classification = Y[0]
            return node

        if(len(vocabulary) == 0):
            #Αν φτάσαμε εδώ χρησιμοποιήσαμε όλες τις λέξεις οπότε δεν γίνονται παραπάνω κατατάξεις.
            #Η διαδικασία σταματάει και γίνεται classified με την τιμή που επικρατεί στα Y
            node = Tree()
            if(Y.count(0)>Y.count(1)):
                max_count = 0
            else:
                max_count = 1
            node.classification = max_count
            return node

        if (self.depth == self.max_depth):
            #Αν είμαστε εδώ φτάσαμε το max depth του δέντρου
            #Η διαδικασία σταματάει και γίνεται classified με την τιμή που επικρατεί στα Y. 
            #Αν έχουμε ισοπαλία αρνητικών θετικών reviews παίρνουμε το default, δηλαδή αυτή που επικρατούσε στο παραπάνω επίπεδο
            yes = True
            node = Tree()
            if((Y.count(0))>Y.count(1)):
                node.classification = 0
            elif((Y.count(0))<Y.count(1)):
                node.classification = 1
            else:
                node.classification =default
            return node

        #Σταματάει η διαδικασία αν υπερτερεί στα εναπομείναντα reviews είτε το 0 είτε το 1
        if (float(Y.count(1))/float(len(Y))>= 0.95):
            node = Tree()
            node.classification = 1                   
            return node
        
        if(float(Y.count(0))/float(len(Y))>= 0.95):
            node = Tree()
            node.classification = 0
            return node

        #Αποθήκευση του clssification που επικρατεί μέχρι στιγμης ώστε να περασθεί ως default στα παρακάτω επίπεδα
        if(Y.count(1)>Y.count(0)):
            max_count = 1
        else:
            max_count = 0


        best_word = self.most_IG(X, Y, vocabulary) #Εύρεση της λέξης με το μέγιστο Information Gain 
        tree = Tree() #Αρχικοποίηση υπόδεντρου

        #το νεο λεξιλόγιο, χωρίς την λέξη που θα χρσιμοποιηθέι για τον διαχωρισμό σε φύλλα τωρα ωστέ να μην ξαναχρησιμοποιηθεί μετά
        new_vocabulary = vocabulary.copy() 
        new_vocabulary.remove(best_word)
        self.depth += 1 #ενημέρωση του depth

        for zero_or_one in range(2):
            # if(zero_or_one==0):
            #     print(len(new_vocabulary))
            #Oι νέες λίστες reviews, δημιουργούνται 2 για κάθε κατηγορία(Υ και Χ) λόγω της for, μια με τη best_word και μία χωρις 
            x_new = list()
            y_new = list()
            for i in range(len(X)):
                if X[i][best_word] == zero_or_one:
                    x_new.append(X[i])
                    y_new.append(Y[i])
            subtree = self.fit(x_new, y_new, new_vocabulary, max_count)
            subtree.tag = zero_or_one 
            subtree.word = best_word
            tree.new_child(subtree)            
                
        return tree

    def singular_prediction(self, X, tree):
        sub_tree = tree #Αρχικοποίηση Υπόδεντρου
        flag = False
        while not flag:
            word_feature = sub_tree.children[0].word #Παίρνουμε τη λέξη με την οποία έγινε ο διαχωρισμός
            for sub in sub_tree.children:
                if (sub.tag == X[word_feature]): 
                    #Αν υπάρχει η λέξη-κριτήριο με την οποία έγινε ο διαχωρισμος σε αυτο το επίπεδο στο sample που κοιταμε 
                    # πάμε στο υπόδεντρο οπόυ το tag είναι 1(δηλ. έχει reviews που την περιλαμβανουν), αλλιώς σε αυτο που ειναι 0
                    sub_tree = sub
            if(sub_tree.classification == 1 or sub_tree.classification == 0):
                #Σταματάμε αν φτάσουμε σε κάποιο φυλλο. Τα φύλλα έχουν τιμή 0 ή 1 και αυτο καταλήγει να ειναι το classification
                # του sample. Οι άλλοι ενδιάμεσοι κόμβοι έχουν None στο classification
                flag = True
        return sub_tree.classification

    def predict(self, tree, X):
        y_pred = list()
        for i in range(len(X)):
            y_pred.append(self.singular_prediction(X[i], tree)) #πρόβλεψη για κάθε review ξεχωριστά
        
        return y_pred
    

## Random Forest classifier

In [9]:
import random

class Random_Forest():
    def __init__(self, num_of_words, trees = 10):
        self.num_of_words = num_of_words #Αριθμός των λέξεων
        self.trees = trees #Αριθμός των δέντρων που θα φτιαχθούν
        self.forest = list() #Λίστα Δέντων

    def new_sample(self, X, Y):
        #Αρχικοποίηση των νέων x και y
        x_new = list()
        y_new = list()

        y_indexes = list() #Tα indexes των reviews που δεν έχουν επιλεχθεί
        for i in range(len(Y)):
            y_indexes.append(i)

        for i in range(len(X)):
            #Τυχαία επιλογή reviews για το υποσύνολο που θα επιστρέψει η μέθοδος, χρησιμοποιώντας τα indexes που φτιάχτηκε πάνω
            random_choice = random.choice(y_indexes) 
            x_new.append(X[random_choice])
            y_new.append(Y[random_choice])

        return x_new, y_new

    def new_vocabulary(self, X):
        #Λίστα με τα indexes του λεξιλογιου για τυχαία επιλογή των νέων λέξεων του νέου λεξιλογίου που επιστρέφει η μέθοδος 
        words_indexes = list()
        for x in range(len(X[0])):
            words_indexes.append(x)

        new_words = list()
        for i in range(self.num_of_words):
            random_word = random.choice(words_indexes) #Tυχαία επιλογή λέξης
            words_indexes.remove(random_word) #Αφαίρεση από το παλιό λεξιλόγιο
            new_words.append(random_word) #Εισαγωγή στο καινούριο

        return new_words

    def fit(self, X, Y, max_depth = 10):
        for i in range(self.trees):
            id3 = ID3(max_depth) #Δημιουργία id3 δέντρου
            random_x, random_y = self.new_sample(X, Y)
            tree = id3.fit(random_x, random_y, self.new_vocabulary(random_x), 0)
            self.forest.append(tree)

    def predict(self, X):
        y_pred = list()
        for i in range(len(X)):
            zeros =0
            ones = 0
            for j in range(self.trees):
                id3 = ID3()
                prediction = id3.singular_prediction(X[i], self.forest[j])
                if (prediction == 1):
                    ones += 1
                elif(prediction==0):
                    zeros +=1
            if ones>zeros:
                y_pred.append(1)
            else:
                y_pred.append(0)
        return y_pred